#### 챗봇 예제(Gradio + csv 사용)

In [1]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage   # HumanMessage: 사용자가 보낸 메시지, AIMessage : LLM의 메시지
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter  # 특정 문자(예: 줄바꿈, 공백)를 기준으로 텍스트를 분할하는 기능을 제공
from langchain.chains import ConversationalRetrievalChain  # 질문과 관련된 정보 검색, 이전 대화정보도 같이 LLM에 제공, 답변생성
import gradio as gr

c:\Python310\aiproject\llm2_Study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CSV 파일 로드
df = pd.read_csv("dataset/indata_kor.csv", encoding="CP949")

In [3]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200) # 텍스트 조각 최대 1000자, 텍스트 조각 사이에 200자 만큼의 중복을 허용(문맥 유지)
texts = text_splitter.split_text("\n".join(df.to_string()))  # 문자열들을 줄바꿈 문자(\n) 기준으로 연결

In [4]:
# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
# 모델 이름 : 조직이름(sentence-transformers) 다양한 작업 가능(all)-MS사 경령화 트랜스포머모델(MiniLM)-모델의 레이어수(L6)-모델이 버전(v2)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\yj\AppData\Local\Temp\ipykernel_13892\3398256464.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:

# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)   # from_texts : 임베딩으로 변환된 벡터를 FAISS 인덱스에 저장

In [6]:
# chatOllama 모델 초기화
llm = ChatOllama(model="gemma2", temperature=0.1)   # temperture가 낮을수록 거의 동일답변, 높을수록 창의적인 답변

C:\Users\yj\AppData\Local\Temp\ipykernel_13892\4166537660.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma2", temperature=0.1)   # temperture가 낮을수록 거의 동일답변, 높을수록 창의적인 답변


In [7]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k":1}),  # 가장 관련성 높은 1개의 문서만 검색
    return_source_documents=True,   # 참고한 소스 문서 정보 반환 여부
    verbose=False   # 체인의 실행 과정 출력 여부
)

In [8]:
# 채팅 함수 정의
def chat(message, history):
    # 이전 대화 기록을 ConversationalRetrievalCfhain 형식으로 변환
    chat_history = [(human, ai) for human, ai in history]
    
    # 모델을 사용하여 응답 생성
    response = qa_chain({"question": message, "chat_history": chat_history})
    
    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else " "
    
    return response['answer'] + source_info

In [9]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title="대학 정보 AI 챗봇",
    description="스마트금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

c:\Python310\aiproject\llm2_Study\.venv\lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [10]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


In [12]:
demo.close()

Closing server running on port: 7861
